In [6]:
!pip install keras -qq --upgrade
from utils import check_keras_3
check_keras_3()

Keras version: 3.0.0


In [1]:
import sys
sys.path.append('../')

In [2]:
import keras
import numpy as np

2023-12-05 13:29:49.808614: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 13:29:49.834513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 13:29:50.232973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
batch_size = 128
epochs = 2
def train_model(model):
    model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [28]:
from k3im.vit import ViT
model = ViT(
    image_size=(28, 28),
    patch_size=(7, 7),
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=8,
    mlp_dim=65,
    channels=1,
    dim_head=32,
    pool="mean",
)

In [29]:
model.summary()

Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_7   │ (None, 4, 4, 49)  │       0 │ input_layer_25[0][0] │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_7 (Reshape) │ (None, 16, 49)    │       0 │ extract_patches_7[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_7[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_50 (Dense)    │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_50[0][0]       │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ class_token_positi… │ (None, 17, 32)    │     576 │ layer_normalization… │
│ (ClassTokenPositio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 17, 32)    │  33,568 │ class_token_positio… │
│ (MultiHeadAttentio… │                   │         │ class_token_positio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_36 (Add)        │ (None, 17, 32)    │       0 │ class_token_positio… │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_18       │ (None, 17, 32)    │   4,321 │ add_36[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_37 (Add)        │ (None, 17, 32)    │       0 │ add_36[0][0],        │
│                     │                   │         │ sequential_18[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 17, 32)    │  33,568 │ add_37[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_37[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_38 (Add)        │ (None, 17, 32)    │       0 │ add_37[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_19       │ (None, 17, 32)    │   4,321 │ add_38[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_39 (Add)        │ (None, 17, 32)    │       0 │ add_38[0][0],        │
│                     │                   │         │ sequential_19[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 17, 32)    │      64 │ add_39[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pool            │ (None, 32)        │       0 │ layer_normalization… │
│ (GlobalAveragePool… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 78,510 (306.68 KB)

 Trainable params: 78,510 (306.68 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9699 - loss: 0.0960 - val_accuracy: 0.9673 - val_loss: 0.1160
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9757 - loss: 0.0773 - val_accuracy: 0.9677 - val_loss: 0.1077
Test loss: 0.11437059938907623
Test accuracy: 0.9635999798774719


In [36]:
from k3im.simple_vit import SimpleViT
model = SimpleViT(
    image_size=(28, 28),
    patch_size=(7, 7),
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=8,
    mlp_dim=65,
    channels=1,
    dim_head=32,
    pool="mean",
)

In [37]:
model.summary()

Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_28      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_8   │ (None, 4, 4, 49)  │       0 │ input_layer_28[0][0] │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_8 (Reshape) │ (None, 16, 49)    │       0 │ extract_patches_8[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_8[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_56 (Dense)    │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_56[0][0]       │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_40 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 16, 32)    │  33,568 │ add_40[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_40[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_41 (Add)        │ (None, 16, 32)    │       0 │ add_40[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_20       │ (None, 16, 32)    │   4,321 │ add_41[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_42 (Add)        │ (None, 16, 32)    │       0 │ add_41[0][0],        │
│                     │                   │         │ sequential_20[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 16, 32)    │  33,568 │ add_42[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_42[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_43 (Add)        │ (None, 16, 32)    │       0 │ add_42[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_21       │ (None, 16, 32)    │   4,321 │ add_43[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_44 (Add)        │ (None, 16, 32)    │       0 │ add_43[0][0],        │
│                     │                   │         │ sequential_21[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ add_44[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ avg_pool            │ (None, 32)        │       0 │ layer_normalization… │
│ (GlobalAveragePool… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_61 (Dense)    │ (None, 10)        │     330 │ avg_pool[0][0]     

 Total params: 77,934 (304.43 KB)

 Trainable params: 77,934 (304.43 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.5450 - loss: 1.3601 - val_accuracy: 0.8840 - val_loss: 0.3747
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8759 - loss: 0.3967 - val_accuracy: 0.9345 - val_loss: 0.2210
Test loss: 0.25774914026260376
Test accuracy: 0.9208999872207642


In [45]:
from k3im.cct import CCT


model = CCT(
    input_shape=input_shape,
    num_heads=8,
    projection_dim=32,
    kernel_size=3,
    stride=3,
    padding=2,
    transformer_units=[16, 32],
    stochastic_depth_rate=0.6,
    transformer_layers=2,
    num_classes=num_classes,
    positional_emb=False,
)

In [46]:
model.summary()

Model: "functional_48"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_39      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ cct_tokenizer_4     │ (None, 9, 32)     │  19,008 │ input_layer_39[0][0] │
│ (CCTTokenizer)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ cct_tokenizer_4[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 9, 32)     │  33,568 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_13 │ (None, 9, 32)     │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_58 (Add)        │ (None, 9, 32)     │       0 │ stochastic_depth_13… │
│                     │                   │         │ cct_tokenizer_4[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ add_58[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_76 (Dense)    │ (None, 9, 16)     │     528 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_42          │ (None, 9, 16)     │       0 │ dense_76[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_77 (Dense)    │ (None, 9, 32)     │     544 │ dropout_42[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_43          │ (None, 9, 32)     │       0 │ dense_77[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_14 │ (None, 9, 32)     │       0 │ dropout_43[0][0]     │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_59 (Add)        │ (None, 9, 32)     │       0 │ stochastic_depth_14… │
│                     │                   │         │ add_58[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ add_59[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 9, 32)     │  33,568 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_15 │ (None, 9, 32)     │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_60 (Add)        │ (None, 9, 32)     │       0 │ stochastic_depth_15… │
│                     │                   │         │ add_59[0][0]         │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 88,971 (347.54 KB)

 Trainable params: 88,971 (347.54 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6220 - loss: 1.1184 - val_accuracy: 0.9267 - val_loss: 0.2526
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9243 - loss: 0.2497 - val_accuracy: 0.9522 - val_loss: 0.1670
Test loss: 0.179606631398201
Test accuracy: 0.9430000185966492


In [7]:
from k3im.convmixer import ConvMixer # Check convmixer


model = ConvMixer(
    image_size=28, filters=64, depth=8, kernel_size=3, patch_size=2, num_classes=10, num_channels=1
)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ rescaling_1         │ (None, 28, 28, 1) │       0 │ input_layer_1[0][0]  │
│ (Rescaling)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 14, 14,    │     320 │ rescaling_1[0][0]    │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_17       │ (None, 14, 14,    │       0 │ conv2d_9[0][0]       │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_17[0][0]  │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ depthwise_conv2d_8  │ (None, 14, 14,    │     640 │ batch_normalization… │
│ (DepthwiseConv2D)   │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_18       │ (None, 14, 14,    │       0 │ depthwise_conv2d_8[… │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_18[0][0]  │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_8 (Add)         │ (None, 14, 14,    │       0 │ batch_normalization… │
│                     │ 64)               │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 14, 14,    │   4,160 │ add_8[0][0]          │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_19       │ (None, 14, 14,    │       0 │ conv2d_10[0][0]      │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_19[0][0]  │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ depthwise_conv2d_9  │ (None, 14, 14,    │     640 │ batch_normalization… │
│ (DepthwiseConv2D)   │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_20       │ (None, 14, 14,    │       0 │ depthwise_conv2d_9[… │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_20[0][0]  │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_9 (Add)         │ (None, 14, 14,    │       0 │ batch_normalization… │
│                     │ 64)               │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 14, 14,    │   4,160 │ add_9[0][0]        

 Total params: 43,722 (170.79 KB)

 Trainable params: 41,546 (162.29 KB)

 Non-trainable params: 2,176 (8.50 KB)

In [8]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 85s 188ms/step - accuracy: 0.8303 - loss: 0.6708 - val_accuracy: 0.0995 - val_loss: 5.8338
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 77s 183ms/step - accuracy: 0.9840 - loss: 0.0601 - val_accuracy: 0.0992 - val_loss: 16.3203
Test loss: 16.400421142578125
Test accuracy: 0.0982000008225441


In [5]:
from k3im.eanet import EANet
model = EANet(
    input_shape=input_shape,
    patch_size=7,
    embedding_dim=64,
    num_transformer_blocks=2,
    mlp_dim=32,
    num_heads=16,
    dim_coefficient=2,
    attention_dropout=0.5,
    projection_dropout=0.5,
    num_classes=10,
)

2023-12-05 13:18:52.363480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-05 13:18:52.383527: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patch_extract       │ (None, None, 49)  │       0 │ input_layer[0][0]    │
│ (PatchExtract)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patch_embedding     │ (None, 16, 64)    │   4,224 │ patch_extract[0][0]  │
│ (PatchEmbedding)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalization │ (None, 16, 64)    │     128 │ patch_embedding[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_1 (Dense)     │ (None, 16, 128)   │   8,320 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape (Reshape)   │ (None, 16, 32, 4) │       0 │ dense_1[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose           │ (None, 32, 16, 4) │       0 │ reshape[0][0]        │
│ (Transpose)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_2 (Dense)     │ (None, 32, 16,    │     160 │ transpose[0][0]      │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ softmax (Softmax)   │ (None, 32, 16,    │       0 │ dense_2[0][0]        │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda (Lambda)     │ (None, 32, 16,    │       0 │ softmax[0][0]        │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout (Dropout)   │ (None, 32, 16,    │       0 │ lambda[0][0]         │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_3 (Dense)     │ (None, 32, 16, 4) │     132 │ dropout[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_1         │ (None, 16, 32, 4) │       0 │ dense_3[0][0]        │
│ (Transpose)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_1 (Reshape) │ (None, 16, 128)   │       0 │ transpose_1[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_4 (Dense)     │ (None, 16, 64)    │   8,256 │ reshape_1[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_1 (Dropout) │ (None, 16, 64)    │       0 │ dense_4[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 16, 64)    │       0 │ dropout_1[0][0],     │
│                     │                   │         │ patch_embedding[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 64)    │     128 │ add[0][0]            │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_5 (Dense)     │ (None, 16, 32)    │   2,080 │ layer_normalization

 Total params: 47,506 (185.57 KB)

 Trainable params: 47,506 (185.57 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step - accuracy: 0.4281 - loss: 1.6386 - val_accuracy: 0.8197 - val_loss: 0.5651
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 14s 34ms/step - accuracy: 0.7678 - loss: 0.7132 - val_accuracy: 0.9038 - val_loss: 0.3027
Test loss: 0.35680198669433594
Test accuracy: 0.8906999826431274


In [11]:
from k3im.gmlp import gMLPModel
model = gMLPModel(
    image_size=28,
    patch_size=7,
    embedding_dim=32,
    num_blocks=4,
    dropout_rate=0.5,
    num_classes=num_classes,
    positional_encoding=False,
    num_channels=1,
)

In [12]:
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ patches_1 (Patches)             │ (None, 16, 49)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_28 (Dense)                │ (None, 16, 32)            │      1,600 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_4 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_5 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_6 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_7 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling1d_2      │ (None, 32)                │          0 │
│ (GlobalAveragePooling1D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_17 (Dropout)            │ (None, 32)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_41 (Dense)                │ (None, 10)                │        330 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 16,202 (63.29 KB)

 Trainable params: 16,202 (63.29 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.2368 - loss: 2.1155 - val_accuracy: 0.7885 - val_loss: 0.7176
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6239 - loss: 1.0990 - val_accuracy: 0.9083 - val_loss: 0.3072
Test loss: 0.3753221035003662
Test accuracy: 0.8913999795913696


In [18]:
from k3im.mlp_mixer import MixerModel
model = MixerModel(
    image_size=28,
    patch_size=7,
    embedding_dim=32,
    num_blocks=4,
    dropout_rate=0.5,
    num_classes=num_classes,
    positional_encoding=True,
    num_channels=1,
)

model.summary()

Model: "functional_43"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_29      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patches_4 (Patches) │ (None, 16, 49)    │       0 │ input_layer_29[0][0] │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_78 (Dense)    │ (None, 16, 32)    │   1,600 │ patches_4[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ position_embedding… │ (None, 16, 32)    │     512 │ dense_78[0][0]       │
│ (PositionEmbedding) │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_6 (Add)         │ (None, 16, 32)    │       0 │ dense_78[0][0],      │
│                     │                   │         │ position_embedding_… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_8   │ (None, 16, 32)    │   1,680 │ add_6[0][0]          │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_9   │ (None, 16, 32)    │   1,680 │ mlp_mixer_layer_8[0… │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_10  │ (None, 16, 32)    │   1,680 │ mlp_mixer_layer_9[0… │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_11  │ (None, 16, 32)    │   1,680 │ mlp_mixer_layer_10[… │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ global_average_poo… │ (None, 32)        │       0 │ mlp_mixer_layer_11[… │
│ (GlobalAveragePool… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_44          │ (None, 32)        │       0 │ global_average_pool… │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_95 (Dense)    │ (None, 10)        │     330 │ dropout_44[0][0]     │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 9,162 (35.79 KB)

 Trainable params: 9,162 (35.79 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.2339 - loss: 2.1750 - val_accuracy: 0.8117 - val_loss: 0.5949
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7022 - loss: 0.8816 - val_accuracy: 0.8948 - val_loss: 0.3417
Test loss: 0.4032406806945801
Test accuracy: 0.8794000148773193


In [22]:
from k3im.simple_vit_with_fft import SimpleViTFFT
model = SimpleViTFFT(image_size=28, patch_size=7, freq_patch_size=7, num_classes=num_classes, dim=32, depth=2, 
                     heads=8, mlp_dim=64, channels=1, 
                     dim_head = 16)
model.summary()

Model: "functional_59"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_48      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_4 (Lambda)   │ (None, 28, 28, 2) │       0 │ input_layer_48[0][0] │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_4   │ (None, 4, 4, 49)  │       0 │ input_layer_48[0][0] │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_5   │ (None, 4, 4, 98)  │       0 │ lambda_4[0][0]       │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_8 (Reshape) │ (None, 16, 49)    │       0 │ extract_patches_4[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_9 (Reshape) │ (None, 16, 98)    │       0 │ extract_patches_5[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_8[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 98)    │     196 │ reshape_9[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_118 (Dense)   │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_119 (Dense)   │ (None, 16, 32)    │   3,168 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_118[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_119[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_27 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_28 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ concatenate_2       │ (None, 32, 32)    │       0 │ add_27[0][0],        │
│ (Concatenate)       │                   │         │ add_28[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 32, 32)    │  16,800 │ concatenate_2[0][0], │
│ (MultiHeadAttentio… │                   │         │ concatenate_2[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_29 (Add)        │ (None, 32, 32)    │       0 │ concatenate_2[0][0], │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_40       │ (None, 32, 32)    │   4,256 │ add_29[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_30 (Add)        │ (None, 32, 32)    │       0 │ add_29[0][0],        │
│                     │                   │         │ sequential_40[0][0]

 Total params: 47,696 (186.31 KB)

 Trainable params: 47,696 (186.31 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.5030 - loss: 1.4358 - val_accuracy: 0.8895 - val_loss: 0.3664
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8832 - loss: 0.3745 - val_accuracy: 0.9370 - val_loss: 0.2077
Test loss: 0.24182891845703125
Test accuracy: 0.9247000217437744


In [25]:
from k3im.simple_vit_with_register_tokens import SimpleViT_RT
model = SimpleViT_RT(image_size=28,
    patch_size=7,
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=4,
    mlp_dim=64,
    num_register_tokens=4,
    channels=1,
    dim_head=64,)

In [26]:
model.summary()

Model: "functional_67"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_54      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_7   │ (None, 4, 4, 49)  │       0 │ input_layer_54[0][0] │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_11          │ (None, 16, 49)    │       0 │ extract_patches_7[0… │
│ (Reshape)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_11[0][0]     │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_131 (Dense)   │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_131[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_38 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ register_tokens_1   │ (None, 20, 32)    │     128 │ add_38[0][0]         │
│ (RegisterTokens)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 20, 32)    │  33,568 │ register_tokens_1[0… │
│ (MultiHeadAttentio… │                   │         │ register_tokens_1[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_39 (Add)        │ (None, 20, 32)    │       0 │ register_tokens_1[0… │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_44       │ (None, 20, 32)    │   4,256 │ add_39[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_40 (Add)        │ (None, 20, 32)    │       0 │ add_39[0][0],        │
│                     │                   │         │ sequential_44[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 20, 32)    │  33,568 │ add_40[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_40[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_41 (Add)        │ (None, 20, 32)    │       0 │ add_40[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_45       │ (None, 20, 32)    │   4,256 │ add_41[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_42 (Add)        │ (None, 20, 32)    │       0 │ add_41[0][0],        │
│                     │                   │         │ sequential_45[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 20, 32)    │      64 │ add_42[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 77,932 (304.42 KB)

 Trainable params: 77,932 (304.42 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.5280 - loss: 1.3909 - val_accuracy: 0.8740 - val_loss: 0.4153
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8705 - loss: 0.4117 - val_accuracy: 0.9178 - val_loss: 0.2573
Test loss: 0.292081743478775
Test accuracy: 0.9110999703407288


In [5]:
from k3im.swint import SwinTModel ########## PROBLEM

In [7]:
model = SwinTModel(
    img_size=28,
    patch_size=7,
    embed_dim=32,
    num_heads=4,
    window_size=4,
    num_mlp=4,
    qkv_bias=True,
    dropout_rate=0.2,
    shift_size=2,
    num_classes=num_classes,
    in_channels=1,
)

In [8]:
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ extract_patches_1               │ (None, 4, 4, 49)          │          0 │
│ (ExtractPatches)                │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_1 (Reshape)             │ (None, 16, 49)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ patch_embedding_1               │ (None, 16, 32)            │      2,112 │
│ (PatchEmbedding)                │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ swin_transformer_2              │ (None, 16, 32)            │      5,096 │
│ (SwinTransformer)               │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ swin_transformer_3              │ (None, 16, 32)            │      5,352 │
│ (SwinTransformer)               │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ patch_merging (PatchMerging)    │ (None, 4, 64)             │      8,192 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling1d        │ (None, 64)                │          0 │
│ (GlobalAveragePooling1D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_19 (Dense)                │ (None, 10)                │        650 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 21,402 (86.60 KB)

 Trainable params: 20,634 (80.60 KB)

 Non-trainable params: 768 (6.00 KB)

In [9]:
train_model(model)

Epoch 1/2


TypeError: Exception encountered when calling Reshape.call().

[1mFailed to convert elements of (None, 16, 49) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.[0m

Arguments received by Reshape.call():
  • x=tf.Tensor(shape=(None, 4, 4, 49), dtype=float32)